In [62]:
# Dependencies
import requests
import json
import quandl
import pandas as pd
from datetime import datetime


In [63]:
date_df = pd.read_csv('../../db/sentences_sample_with_datetime.csv',sep=',')
date_df['call_date'] = pd.to_datetime(date_df['call_date']).dt.date
date_df.head()



,Unnamed: 0,id,sentiment,sentence,ticker,call_title,speaker,call_section,call_datetime,call_date,call_time
0,0,4065,NaN,Patrik's proven command of this business and t...,UAA,"Under Armour, Inc. (NYSE:UAA) Q3 2019 Results ...",Kevin Plank,presentation,2019-11-04 08:30:00,2019-11-04,08:30:00
1,1,2730,NaN,"We will be at the Jefferies conference, hardwa...",NVDA,NVIDIA Corporation (NASDAQ:NVDA) Q2 2020 Earni...,Colette Kress,presentation,2019-08-15 17:30:00,2019-08-15,17:30:00
2,2,4044,NaN,[Operator Instructions].,UAA,"Under Armour, Inc. (NYSE:UAA) Q3 2019 Results ...",Operator,operator_instruction,2019-11-04 08:30:00,2019-11-04,08:30:00
3,3,1833,NaN,Any update that you can share in terms of mile...,GM,General Motors Company (NYSE:GM) Q3 2019 Earni...,Itay Michaeli,question,2019-10-29 10:00:00,2019-10-29,10:00:00
4,4,2721,NaN,"With that, let me turn to the outlook for the ...",NVDA,NVIDIA Corporation (NASDAQ:NVDA) Q2 2020 Earni...,Colette Kress,presentation,2019-08-15 17:30:00,2019-08-15,17:30:00


In [64]:
price_df = pd.read_csv('../../db/ticker_closing_prices.csv',sep=',')
price_df['Date'] = pd.to_datetime(price_df['Date']).dt.date
price_df.head()


,Date,High,Low,Open,Close,Volume,Adj Close,ticker
0,2019-01-02,17.040001,16.320000,16.420000,16.840000,5222300.0,15.740013,IVZ
1,2019-01-03,16.850000,16.360001,16.750000,16.430000,6000500.0,15.356794,IVZ
2,2019-01-04,17.340000,16.620001,16.750000,17.260000,5041500.0,16.132580,IVZ
3,2019-01-07,17.719999,17.010000,17.190001,17.469999,3884700.0,16.328859,IVZ
4,2019-01-08,17.820000,17.190001,17.600000,17.280001,3998100.0,16.151274,IVZ


In [65]:
merged_df = date_df.merge(price_df, left_on=['ticker','call_date'], right_on = ['ticker','Date'], how='inner')
merged_df

,Unnamed: 0,id,sentiment,sentence,ticker,call_title,speaker,call_section,call_datetime,call_date,call_time,Date,High,Low,Open,Close,Volume,Adj Close
0,0,4065,NaN,Patrik's proven command of this business and t...,UAA,"Under Armour, Inc. (NYSE:UAA) Q3 2019 Results ...",Kevin Plank,presentation,2019-11-04 08:30:00,2019-11-04,08:30:00,2019-11-04,18.510000,16.830000,17.830000,17.139999,43419100.0,17.139999
1,2,4044,NaN,[Operator Instructions].,UAA,"Under Armour, Inc. (NYSE:UAA) Q3 2019 Results ...",Operator,operator_instruction,2019-11-04 08:30:00,2019-11-04,08:30:00,2019-11-04,18.510000,16.830000,17.830000,17.139999,43419100.0,17.139999
2,14,4590,NaN,And I believe that we started that.,UAA,"Under Armour, Inc. (NYSE:UAA) Q3 2019 Results ...",Kevin Plank,answer,2019-11-04 08:30:00,2019-11-04,08:30:00,2019-11-04,18.510000,16.830000,17.830000,17.139999,43419100.0,17.139999
3,25,4671,NaN,"It's tough getting to $5 billion, I'll tell yo...",UAA,"Under Armour, Inc. (NYSE:UAA) Q3 2019 Results ...",Kevin Plank,answer,2019-11-04 08:30:00,2019-11-04,08:30:00,2019-11-04,18.510000,16.830000,17.830000,17.139999,43419100.0,17.139999
4,30,4399,NaN,"I know you're not guiding to 2020, but just ma...",UAA,"Under Armour, Inc. (NYSE:UAA) Q3 2019 Results ...",Edward Yruma,question,2019-11-04 08:30:00,2019-11-04,08:30:00,2019-11-04,18.510000,16.830000,17.830000,17.139999,43419100.0,17.139999
5,40,4063,NaN,I'd like to start today's call by touching on ...,UAA,"Under Armour, Inc. (NYSE:UAA) Q3 2019 Results ...",Kevin Plank,presentation,2019-11-04 08:30:00,2019-11-04,08:30:00,2019-11-04,18.510000,16.830000,17.830000,17.139999,43419100.0,17.139999
6,49,4155,NaN,"SG&A expense increased 4% to $551 million, whi...",UAA,"Under Armour, Inc. (NYSE:UAA) Q3 2019 Results ...",David Bergman,presentation,2019-11-04 08:30:00,2019-11-04,08:30:00,2019-11-04,18.510000,16.830000,17.830000,17.139999,43419100.0,17.139999
7,56,4383,NaN,"So, you're seeing a bit of a shift in terms of...",UAA,"Under Armour, Inc. (NYSE:UAA) Q3 2019 Results ...",Patrik Frisk,answer,2019-11-04 08:30:00,2019-11-04,08:30:00,2019-11-04,18.510000,16.830000,17.830000,17.139999,43419100.0,17.139999
8,64,4676,NaN,Thank you very much.,UAA,"Under Armour, Inc. (NYSE:UAA) Q3 2019 Results ...",Kevin Plank,answer,2019-11-04 08:30:00,2019-11-04,08:30:00,2019-11-04,18.510000,16.830000,17.830000,17.139999,43419100.0,17.139999
9,78,4421,NaN,"So, today, we stick with roughly four, we call...",UAA,"Under Armour, Inc. (NYSE:UAA) Q3 2019 Results ...",Kevin Plank,answer,2019-11-04 08:30:00,2019-11-04,08:30:00,2019-11-04,18.510000,16.830000,17.830000,17.139999,43419100.0,17.139999
